# Architecture - Semantic Search using Elastic Search

![ElasticSearch-Semantic Search Architecture](./es_semantic_search.png)

- Two very important concepts in Elasticsearch are documents and indexes.

- A document is collection of fields with their associated values.

- To work with Elasticsearch you have to organize your data into documents, and then add all your documents to an index.

- Index as a collection of documents that is stored in a highly optimized format designed to perform efficient searches.

## Step 1: Prepare documents

In [1]:
import json

with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [2]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [3]:
documents[:3]

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines 

In [4]:
len(documents)

948

## Step 2: Create Embeddings using Pretrained Models
Sentence Transformers documentation here: https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

In [6]:
# This is a new library compared to the previous modules. 
# Please perform "pip install sentence_transformers==2.7.0"
from sentence_transformers import SentenceTransformer

# if you get an error do the following:
# 1. Uninstall numpy 
# 2. Uninstall torch
# 3. pip install numpy==1.26.4
# 4. pip install torch
# run the above cell, it should work
model = SentenceTransformer("all-mpnet-base-v2")

In [7]:
model.encode("This is a simple sentence")

array([ 4.44866112e-03, -7.61314556e-02, -3.77446471e-04,  7.52516696e-03,
       -3.80979329e-02,  3.80131006e-02, -9.73009970e-03, -5.05399844e-03,
       -9.37975291e-03,  1.23888394e-02,  4.91277501e-02,  1.52208796e-02,
        3.80008630e-02, -6.41803071e-02,  9.42132436e-03, -5.19748665e-02,
        9.08066630e-02,  1.71115026e-02,  1.62125062e-02,  2.98866685e-02,
        1.50536583e-03,  8.35076347e-03,  3.78842577e-02, -1.01191979e-02,
        6.46103034e-03,  3.97758231e-05, -1.45218093e-02, -1.88468620e-02,
       -3.74039710e-02, -1.51659688e-03, -1.02681294e-02, -3.68062370e-02,
        2.36677583e-02, -6.46022633e-02,  1.96967085e-06, -5.01107238e-03,
       -2.80825864e-03, -1.92073677e-02, -8.65119845e-02,  2.83464324e-02,
       -5.38665913e-02,  3.63704897e-02, -2.26467345e-02,  2.87367441e-02,
       -1.32342046e-02,  1.08689591e-01,  3.70519422e-02,  3.38801071e-02,
       -5.30680269e-02,  3.61782536e-02, -1.35722244e-03, -3.63483354e-02,
       -2.78346483e-02, -

In [8]:
len(model.encode("This is a simple sentence"))

768

In [9]:
documents[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp'}

In [10]:
#created the dense vector using the pre-trained model
operations = []
for doc in documents:
    # Transforming the document text into an embedding using the model
    doc["text_vector"] = model.encode(doc["text"]).tolist()
    operations.append(doc)

In [12]:
operations[1]

{'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
 'section': 'General course-related questions',
 'question': 'Course - What are the prerequisites for this course?',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.04103028401732445,
  0.025834180414676666,
  -0.03680187836289406,
  -0.020898321643471718,
  -0.020596222952008247,
  0.00935377087444067,
  -0.0033316484186798334,
  -0.009491927921772003,
  0.03011787123978138,
  0.01908211037516594,
  0.012690058909356594,
  -0.01707880012691021,
  -0.0016323638847097754,
  0.12997247278690338,
  0.03096924163401127,
  -0.0258235614746809,
  0.02782302536070347,
  0.02515980787575245,
  -0.08081213384866714,
  -0.003617374924942851,
  -0.008901946246623993,
  0.003404894843697548,
  -0.023009328171610832,
  -0.03404533490538597,
  0.024598650634288788,
  0.013545590452849865,
  -0.025438902899622917,
  0.011951095424592495,
  -0.020540133118629456,
  -0.010077448561787605,
  0.020575400441884995,
  0.

# Step 3: Setup ElasticSearch connection

In [11]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200') 

es_client.info()

ObjectApiResponse({'name': '41ab304cbf63', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'Hh5hYkEKT46aeWjuC6VYdg', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [15]:
def delete_es_index(index_name):
    # Delete the index
    try:
        response = es_client.indices.delete(index=index_name, ignore_unavailable=True)
        print(f"Index '{index_name}' deleted successfully: {response}")
    except Exception as e:
        print(f"Error deleting index '{index_name}': {e}")


# delete_es_index(index_name)

# Step 4: Create Mappings and Index
- Mapping is the process of defining how a document, and the fields it contains, are stored and indexed.

- Each document is a collection of fields, which each have their own data type.

- We can compare mapping to a database schema in how it describes the fields and properties that documents hold, the datatype of each field (e.g., string, integer, or date), and how those fields should be indexed and stored

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} ,
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"},
        }
    }
}

In [16]:
index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

## Step 5: Add documents into index

In [17]:
for doc in operations:
    try:
        es_client.index(index=index_name, document=doc)
    except Exception as e:
        print(e)

## Step 6: Create end user query

In [18]:
search_term = "windows or mac?"
vector_search_term = model.encode(search_term)

In [19]:
query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000, 
}

In [20]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'RqXXpJABw61zLEwSNxpI',
  '_score': 0.7147919,
  '_source': {'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully'}},
 {'_index': 'course-questions',
  '_id': 'WaXXpJABw61zLEwSUh0n',
  '_score': 0.61347365,
  '_source': {'question': 'WSL instructions',
   'course': 'mlops-zoomcamp',
   'section': 'Module 1: Introduction',
   'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares a

### Testing using a search query (Keyword Search)

In [21]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [23]:
query = "windows or python"

results = elastic_search(query)

In [25]:
results[0]

{'text': 'Problem: If you have already installed pgcli but bash doesn\'t recognize pgcli\nOn Git bash: bash: pgcli: command not found\nOn Windows Terminal: pgcli: The term \'pgcli\' is not recognized…\nSolution: Try adding a Python path C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts to Windows PATH\nFor details:\nGet the location: pip list -v\nCopy C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\site-packages\n3. Replace site-packages with Scripts: C:\\Users\\...\\AppData\\Roaming\\Python\\Python39\\Scripts\nIt can also be that you have Python installed elsewhere.\nFor me it was under c:\\python310\\lib\\site-packages\nSo I had to add c:\\python310\\lib\\Scripts to PATH, as shown below.\nPut the above path in "Path" (or "PATH") in System Variables\nReference: https://stackoverflow.com/a/68233660',
 'section': 'Module 1: Docker and Terraform',
 'question': 'PGCLI - pgcli: command not found',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.024554720148444176,

In [26]:
results[-1]

{'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
 'section': 'Module 6: streaming with kafka',
 'question': 'Module “kafka” not found when trying to run producer.py',
 'course': 'data-engineering-zoomcamp',
 'text_vector': [-0.004933493677526712,
  0.0009792977944016457,
  -0.01875283382833004,
  -0.06503251940011978,
  0.017738735303282738,
  0.022090541198849678,
  -0.02824445627629757,
  -0.02

In [27]:
len(results)

5

In [28]:
## Step 7: Perform Keyword search with Semantic Search (Hybrid/Advanced Search)

# Note: minor modification to the query shown in the notebook here above to use vector embeddings for semantic Search
# Included "knn" in the search query (to perform a semantic search) along with the filter  
search_term = "windows or python"
vector_search_term = model.encode(search_term)

knn_query = {
    "field": "text_vector",
    "query_vector": vector_search_term,
    "k": 5,
    "num_candidates": 10000
}

In [29]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "data-engineering-zoomcamp"},
    },
    knn=knn_query,
    size=5
)

In [31]:
response["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'RqXXpJABw61zLEwSNxpI',
  '_score': 1.4868538,
  '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
   'section': 'General course-related questions',
   'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
   'course': 'data-engineering-zoomcamp',
   'text_vector': [-0.02696545608341694,
    -0.0006260281661525369,
    -0.016629504039883614,
    0.052851416170597076,
    0.054765354841947556,
    -0.0313398540019989,
    0.029942601919174194,
    -0.04808562994003296,
    0.04467545822262764,
    0.005839481949806213,
    0.016233112663030624,
    0.01200118474662304,
    -0.031222324818372726,
    0.01660057343542576,
    -0.04886900633573532,
    -0.06496299058198929,
    0.04643424227833748,
    -0.009297666139900684,
    -0.06425285339355469,
    -0.01373268011957407,
    -0.015976225957274437,
    0.008629551157355309,
    -0.02447894588112

In [32]:
response["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'RqXXpJABw61zLEwSNxpI',
 '_score': 1.4868538,
 '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
  'section': 'General course-related questions',
  'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.02696545608341694,
   -0.0006260281661525369,
   -0.016629504039883614,
   0.052851416170597076,
   0.054765354841947556,
   -0.0313398540019989,
   0.029942601919174194,
   -0.04808562994003296,
   0.04467545822262764,
   0.005839481949806213,
   0.016233112663030624,
   0.01200118474662304,
   -0.031222324818372726,
   0.01660057343542576,
   -0.04886900633573532,
   -0.06496299058198929,
   0.04643424227833748,
   -0.009297666139900684,
   -0.06425285339355469,
   -0.01373268011957407,
   -0.015976225957274437,
   0.008629551157355309,
   -0.02447894588112831,
   -0.005980647634714842,

In [33]:
response["hits"]["hits"][1]

{'_index': 'course-questions',
 '_id': 'NqXXpJABw61zLEwSNhqO',
 '_score': 1.4621526,
 '_source': {'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
  'section': 'General course-related questions',
  'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.008795924484729767,
   0.018822649493813515,
   0.008203222416341305,
   -0.024633318185806274,
   0.029965581372380257,
   0.020212220028042793,
   0.1123519018292427,
   0.008089442737400532,
   0.006306787021458149,
   -0.020710265263915062,
   0.009332895278930664,
   0.013378201983869076,
   -0.026092801243066788,
   0.052493512630462646,
   -0.017481127753853798,
   -0.06065764278173447,
   0.024950087070465088,
   -0.00650993874296546,
   0.04337342083454132,
   0.04361394792795181,
   -0.025195900350809097,
   -0.016572894528508186,
   -0

In [34]:
response["hits"]["hits"][2]

{'_index': 'course-questions',
 '_id': 'I6XXpJABw61zLEwSNRog',
 '_score': 0.778914,
 '_source': {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.035703495144844055,
   -0.06891431659460068,
   -0.04448336735367775,
   0.006250118371099234,
   -0.04224714636802673,
   -0.006454017013311386,
   0.02754802815616131,
   -0.018610380589962006,
   -0.052037570625543594,
   -0.0024047044571489096,
   0.05745451

In [35]:
response["hits"]["hits"][3]

{'_index': 'course-questions',
 '_id': 'JKXXpJABw61zLEwSNRph',
 '_score': 0.778914,
 '_source': {'text': 'GitHub - DataTalksClub data-engineering-zoomcamp#prerequisites',
  'section': 'General course-related questions',
  'question': 'Course - What are the prerequisites for this course?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.04103028401732445,
   0.025834180414676666,
   -0.03680187836289406,
   -0.020898321643471718,
   -0.020596222952008247,
   0.00935377087444067,
   -0.0033316484186798334,
   -0.009491927921772003,
   0.03011787123978138,
   0.01908211037516594,
   0.012690058909356594,
   -0.01707880012691021,
   -0.0016323638847097754,
   0.12997247278690338,
   0.03096924163401127,
   -0.0258235614746809,
   0.02782302536070347,
   0.02515980787575245,
   -0.08081213384866714,
   -0.003617374924942851,
   -0.008901946246623993,
   0.003404894843697548,
   -0.023009328171610832,
   -0.03404533490538597,
   0.024598650634288788,
   0.013545590452849865,
  

In [36]:
response["hits"]["hits"][-1]

{'_index': 'course-questions',
 '_id': 'JaXXpJABw61zLEwSNRp9',
 '_score': 0.778914,
 '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.05720662325620651,
   0.019887369126081467,
   -0.02433602884411812,
   -0.010076554492115974,
   0.03898916020989418,
   0.010869265533983707,
   0.026468876749277115,
   -0.03391910344362259,
   0.029106317088007927,
   -0.04912995174527168,
   0.03300593048334122,
   -0.06733984500169754,
   -0.021426698192954063,
   -0.013197527267038822,
   -0.03563873469829559,
   0.06667865067720413,
   -0.04194782301783562,
   -0.05011487752199173,
   -0.05269119143486023,
   -0.04796301946043968,


In [37]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"section": "General course-related questions"},
    },
    knn=knn_query,
    size=5
)

In [38]:
response["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'RqXXpJABw61zLEwSNxpI',
 '_score': 11.607861,
 '_source': {'text': 'Yes! Linux is ideal but technically it should not matter. Students last year used all 3 OSes successfully',
  'section': 'General course-related questions',
  'question': 'Environment - Is the course [Windows/mac/Linux/...] friendly?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.02696545608341694,
   -0.0006260281661525369,
   -0.016629504039883614,
   0.052851416170597076,
   0.054765354841947556,
   -0.0313398540019989,
   0.029942601919174194,
   -0.04808562994003296,
   0.04467545822262764,
   0.005839481949806213,
   0.016233112663030624,
   0.01200118474662304,
   -0.031222324818372726,
   0.01660057343542576,
   -0.04886900633573532,
   -0.06496299058198929,
   0.04643424227833748,
   -0.009297666139900684,
   -0.06425285339355469,
   -0.01373268011957407,
   -0.015976225957274437,
   0.008629551157355309,
   -0.02447894588112831,
   -0.005980647634714842,

In [39]:
response["hits"]["hits"][1]

{'_index': 'course-questions',
 '_id': 'NqXXpJABw61zLEwSNhqO',
 '_score': 11.583159,
 '_source': {'text': 'Yes, for simplicity (of troubleshooting against the recorded videos) and stability. [source]\nBut Python 3.10 and 3.11 should work fine.',
  'section': 'General course-related questions',
  'question': 'Environment - Is Python 3.9 still the recommended version to use in 2024?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.008795924484729767,
   0.018822649493813515,
   0.008203222416341305,
   -0.024633318185806274,
   0.029965581372380257,
   0.020212220028042793,
   0.1123519018292427,
   0.008089442737400532,
   0.006306787021458149,
   -0.020710265263915062,
   0.009332895278930664,
   0.013378201983869076,
   -0.026092801243066788,
   0.052493512630462646,
   -0.017481127753853798,
   -0.06065764278173447,
   0.024950087070465088,
   -0.00650993874296546,
   0.04337342083454132,
   0.04361394792795181,
   -0.025195900350809097,
   -0.016572894528508186,
   -0

In [40]:
response["hits"]["hits"][2]

{'_index': 'course-questions',
 '_id': 'I6XXpJABw61zLEwSNRog',
 '_score': 10.89992,
 '_source': {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.035703495144844055,
   -0.06891431659460068,
   -0.04448336735367775,
   0.006250118371099234,
   -0.04224714636802673,
   -0.006454017013311386,
   0.02754802815616131,
   -0.018610380589962006,
   -0.052037570625543594,
   -0.0024047044571489096,
   0.05745451

In [41]:
response["hits"]["hits"][-1]

{'_index': 'course-questions',
 '_id': 'JaXXpJABw61zLEwSNRp9',
 '_score': 10.89992,
 '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp',
  'text_vector': [-0.05720662325620651,
   0.019887369126081467,
   -0.02433602884411812,
   -0.010076554492115974,
   0.03898916020989418,
   0.010869265533983707,
   0.026468876749277115,
   -0.03391910344362259,
   0.029106317088007927,
   -0.04912995174527168,
   0.03300593048334122,
   -0.06733984500169754,
   -0.021426698192954063,
   -0.013197527267038822,
   -0.03563873469829559,
   0.06667865067720413,
   -0.04194782301783562,
   -0.05011487752199173,
   -0.05269119143486023,
   -0.04796301946043968,


In [42]:
documents[-1]

{'text': 'Problem description\nInfrastructure created in AWS with CD-Deploy Action needs to be destroyed\nSolution description\nFrom local:\nterraform init -backend-config="key=mlops-zoomcamp-prod.tfstate" --reconfigure\nterraform destroy --var-file vars/prod.tfvars\nAdded by Erick Calderin',
 'section': 'Module 6: Best practices',
 'question': 'How to destroy infrastructure created via GitHub Actions',
 'course': 'mlops-zoomcamp',
 'text_vector': [-0.01823153719305992,
  -0.04600517451763153,
  -0.009615903720259666,
  -0.004917514510452747,
  0.03871743008494377,
  -0.003879561787471175,
  -0.01000662799924612,
  0.009894416667521,
  -0.009460185654461384,
  0.037535689771175385,
  -0.03273559734225273,
  0.05963641032576561,
  0.02431623451411724,
  0.10026902705430984,
  -0.0354011170566082,
  0.04226543381810188,
  -0.029118767008185387,
  0.0011962593998759985,
  -0.07810244709253311,
  -0.017459243535995483,
  -0.032734718173742294,
  0.02679167129099369,
  -0.008486900478601456

In [43]:
response = es_client.search(
    index=index_name,
    query={
        "match": {"course": "mlops-zoomcamp"},
    },
    knn=knn_query,
    size=5
)

In [44]:
response["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'WaXXpJABw61zLEwSUh0n',
 '_score': 2.651766,
 '_source': {'text': 'If you wish to use WSL on your windows machine, here are the setup instructions:\nCommand: Sudo apt install wget\nGet Anaconda download address here. wget <download address>\nTurn on Docker Desktop WFree Download | AnacondaSL2\nCommand: git clone <github repository address>\nVSCODE on WSL\nJupyter: pip3 install jupyter\nAdded by Gregory Morris (gwm1980@gmail.com)\nAll in all softwares at one shop:\nYou can use anaconda which has all built in services like pycharm, jupyter\nAdded by Khaja Zaffer (khajazaffer@aln.iseg.ulisboa.pt)\nFor windows “wsl --install” in Powershell\nAdded by Vadim Surin (vdmsurin@gmai.com)',
  'section': 'Module 1: Introduction',
  'question': 'WSL instructions',
  'course': 'mlops-zoomcamp',
  'text_vector': [-0.0709533616900444,
   0.014381347224116325,
   -0.004209131933748722,
   -0.04673800989985466,
   0.053254079073667526,
   0.030075980350375175,
   0.

In [45]:
response["hits"]["hits"][1]

{'_index': 'course-questions',
 '_id': 'TaXXpJABw61zLEwSUR3Y',
 '_score': 1.9245386,
 '_source': {'text': 'MLOps Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course, and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\n[Problem description]\n[Solution description]\n(optional) Added by Name',
  'section': '+-General course questions',
  'question': 'Format for questions: [Problem title]',
  'course': 'mlops-zoomcamp',
  'text_vector': [-0.00917984638363123,
   -0.07367172837257385,
   -0.01257041934877634,
   -0.008170546032488346,
   0.010255626402795315,
   0.034650400280952454,
   -0.019679682329297066,
   -0.027317015454173088,
   0.032853156328201294,
   -0.03081277385354042,
   0.038497619330883026,
   -0.025062592700123787,
   0.027510346844792366,
   0.09733685851097107,
   0.019815528765320778,
   -0.063

In [46]:
response["hits"]["hits"][-1]

{'_index': 'course-questions',
 '_id': 'UKXXpJABw61zLEwSUR3s',
 '_score': 1.9245386,
 '_source': {'text': 'Yes, it will start in May 2024',
  'section': '+-General course questions',
  'question': 'Will there be a 2024 Cohort? When will the 2024 cohort start?',
  'course': 'mlops-zoomcamp',
  'text_vector': [-0.050861477851867676,
   0.09255291521549225,
   -0.017914948984980583,
   0.0070521775633096695,
   0.016179436817765236,
   -0.04329148679971695,
   -0.02446720562875271,
   -0.0284891314804554,
   -0.03777487203478813,
   -0.007216682191938162,
   0.06447342783212662,
   -0.03314254432916641,
   -0.026777200400829315,
   0.06373245269060135,
   0.016819292679429054,
   -0.058672741055488586,
   0.020484289154410362,
   -0.016429249197244644,
   -0.007168980780988932,
   -0.02364458329975605,
   -0.022399215027689934,
   0.0032226284965872765,
   -0.07078644633293152,
   0.013699018396437168,
   -0.036890506744384766,
   -0.016336509957909584,
   -0.06825783103704453,
   -0.0259

## References
- https://logz.io/blog/elasticsearch-mapping/#:~:text=Within%20a%20search%20engine%2C%20mapping,indexes%20and%20stores%20its%20fields

- https://www.sbert.net/docs/sentence_transformer/pretrained_models.html

- https://www.elastic.co/search-labs/tutorials

- https://www.elastic.co/search-labs/blog/text-similarity-search-with-vectors-in-elasticsearch